# RAG Evaluation
## 1. Environment setup

In [ ]:
%pip install -q langchain langchain-nvidia-ai-endpoints gradio rich
%pip install -q arxiv pymupdf faiss-cpu ragas
%pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━

In [ ]:
import os
os.environ["NVIDIA_API_KEY"] = "NVIDIA_API_KE"

In [ ]:
from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
norm_style = Style(bold=True)
pprint = partial(console.print, style=base_style)
pprint2 = partial(console.print, style=norm_style)

from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")

# ChatNVIDIA.get_available_models(base_url="http://llm_client:9000/v1")
instruct_llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct")

## 2. PairWise Evaluater
### 1.  Pull In Your Document Retrieval Index

In [ ]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_community.vectorstores import FAISS

!tar xzvf docstore_index.tgz
docstore = FAISS.load_local("docstore_index", embedder, allow_dangerous_deserialization=True)
docs = list(docstore.docstore._dict.values())

docstore_index/
docstore_index/index.pkl
docstore_index/index.faiss


In [ ]:
def format_chunk(doc):
    return (
        f"Paper: {doc.metadata.get('Title', 'unknown')}"
        f"\n\nSummary: {doc.metadata.get('Summary', 'unknown')}"
        f"\n\nPage Body: {doc.page_content}"
    )

## This printout just confirms that your store has been retrieved
pprint(f"Constructed aggregate docstore with {len(docstore.docstore._dict)} chunks")
pprint(f"Sample Chunk:")
print(format_chunk(docs[len(docs)//2]))

Constructed aggregate docstore with 571 chunks

Sample Chunk:

Paper: ReAct: Synergizing Reasoning and Acting in Language Models

Summary: While large language models (LLMs) have demonstrated impressive capabilities
across tasks in language understanding and interactive decision making, their
abilities for reasoning (e.g. chain-of-thought prompting) and acting (e.g.
action plan generation) have primarily been studied as separate topics. In this
paper, we explore the use of LLMs to generate both reasoning traces and
task-specific actions in an interleaved manner, allowing for greater synergy
between the two: reasoning traces help the model induce, track, and update
action plans as well as handle exceptions, while actions allow it to interface
with external sources, such as knowledge bases or environments, to gather
additional information. We apply our approach, named ReAct, to a diverse set of
language and decision making tasks and demonstrate its effectiveness over
state-of-the-art baselines, as well as improved human interpretability and
trustwor

### 2. Pull In Your RAG Chain

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableBranch
from langchain_core.runnables.passthrough import RunnableAssign
from langchain.document_transformers import LongContextReorder

from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from functools import partial
from operator import itemgetter

import gradio as gr

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")

# ChatNVIDIA.get_available_models()
instruct_llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct")
llm = instruct_llm | StrOutputParser()

def docs2str(docs, title="Document"):
    """Useful utility for making chunks into context string. Optional, but useful"""
    out_str = ""
    for doc in docs:
        doc_name = getattr(doc, 'metadata', {}).get('Title', title)
        if doc_name: out_str += f"[Quote from {doc_name}] "
        out_str += getattr(doc, 'page_content', str(doc)) + "\n"
    return out_str


In [ ]:
chat_prompt = ChatPromptTemplate.from_template(
    "You are a document chatbot. Help the user as they ask questions about documents."
    " User messaged just asked you a question: {input}\n\n"
    " The following information may be useful for your response: "
    " Document Retrieval:\n{context}\n\n"
    " (Answer only from retrieval. Only cite sources that are used. Make your response conversational)"
    "\n\nUser Question: {input}"
)

In [ ]:
def output_puller(inputs):
    """"Output generator. Useful if your chain returns a dictionary with key 'output'"""
    if isinstance(inputs, dict):
        inputs = [inputs]
    for token in inputs:
        if token.get('output'):
            yield token.get('output')


In [ ]:


## Chain 1 Specs: "Hello World" -> retrieval_chain
##   -> {'input': <str>, 'context' : <str>}
long_reorder = RunnableLambda(LongContextReorder().transform_documents)  ## GIVEN
context_getter = RunnableLambda(lambda x: x)
retrieval_chain = {'input' : (lambda x: x)} | RunnableAssign({'context' : context_getter})

##   -> {"output" : <str>, ...} -> output_puller
generator_chain = RunnableLambda(lambda x: x)  ## TODO
generator_chain = {'output' : generator_chain} | RunnableLambda(output_puller)  ## GIVEN


In [ ]:

rag_chain = retrieval_chain | generator_chain

# pprint(rag_chain.invoke("Tell me something interesting!"))
for token in rag_chain.stream("Tell me something interesting!"):
    print(token, end="")

{'input': 'Tell me something interesting!', 'context': {'input': 'Tell me something interesting!'}}

### 3. Generating Synthetic Question-Answer Pairs

In [ ]:
import random

num_questions = 3
synth_questions = []
synth_answers = []

simple_prompt = ChatPromptTemplate.from_messages([('system', '{system}'), ('user', 'INPUT: {input}')])


In [ ]:
for i in range(num_questions):
    doc1, doc2 = random.sample(docs, 2)
    sys_msg = (
        "Use the documents provided by the user to generate an interesting question-answer pair."
        " Try to use both documents if possible, and rely more on the document bodies than the summary."
        " Use the format:\nQuestion: (good question, 1-3 sentences, detailed)\n\nAnswer: (answer derived from the documents)"
        " DO NOT SAY: \"Here is an interesting question pair\" or similar. FOLLOW FORMAT!"
    )
    usr_msg = (
        f"Document1: {format_chunk(doc1)}\n\n"
        f"Document2: {format_chunk(doc2)}"
    )

    qa_pair = (simple_prompt | llm).invoke({'system': sys_msg, 'input': usr_msg})
    synth_questions += [qa_pair.split('\n\n')[0]]
    synth_answers += [qa_pair.split('\n\n')[1]]
    pprint2(f"QA Pair {i+1}")
    pprint2(synth_questions[-1])
    pprint(synth_answers[-1])
    print()

QA Pair 1

Question: How does the ReAct approach address the issues of hallucination and error propagation prevalent in 
chain-of-thought reasoning, and what improvement does it bring to the task of question answering?

Answer: According to the paper, ReAct overcomes issues of hallucination and error propagation prevalent in 
chain-of-thought reasoning by interacting with a simple Wikipedia API, and generates human-like task-solving 
trajectories that are more interpretable than baselines without reasoning traces. Specifically, on question 
answering tasks like HotpotQA, ReAct is able to retrieve up-to-date information from the Internet and provide a 
reasonable answer, whereas other methods like Standard and CoT give wrong answers due to hallucination.

QA Pair 2

Question: Can a model trained on natural language supervision from a massive dataset of image-text pairs learn to 
reason and access various types of knowledge beyond visual recognition, such as mathematical information or current
events?

Answer: The MRKL architecture discussed in Document2 suggests that a model can potentially overcome the limitations
of large language models (LMs) by incorporating discrete knowledge and reasoning modules. However, the performance 
of the model described in Document1, CLIP, is limited to zero-shot transfer and relies heavily on visual 
recognition, with poor performance on fine-grained detection and no evidence of accessing knowledge beyond the 
training data. Nevertheless, the existence of the MRKL architecture and its implementation in Jurassic-X hint at 
the possibility of future advancements in language models, enabling them to access and reason about various types 
of knowledge.

QA Pair 3

Question: How can large language models (LLMs) be utilized to improve their reasoning and acting abilities, 
especially in handling exceptions and interacting with external sources?

Answer: According to the paper "ReAct: Synergizing Reasoning and Acting in Language Models", the approach, named 
ReAct, involves generating both reasoning traces and task-specific actions in an interleaved manner. This allows 
the model to induce, track, and update action plans, handle exceptions, and interface with external sources such as
knowledge bases or environments to gather additional information.

### 4.  Answer The Synthetic Questions

In [ ]:

rag_answers = []
for i, q in enumerate(synth_questions):
    ## TODO: Compute the RAG Answer
    rag_answer = ""
    rag_answers += [rag_answer]
    pprint2(f"QA Pair {i+1}", q, "", sep="\n")
    pprint(f"RAG Answer: {rag_answer}", "", sep='\n')

QA Pair 1
Question: How does the ReAct approach address the issues of hallucination and error propagation prevalent in 
chain-of-thought reasoning, and what improvement does it bring to the task of question answering?

RAG Answer: 

QA Pair 2
Question: Can a model trained on natural language supervision from a massive dataset of image-text pairs learn to 
reason and access various types of knowledge beyond visual recognition, such as mathematical information or current
events?

RAG Answer: 

QA Pair 3
Question: How can large language models (LLMs) be utilized to improve their reasoning and acting abilities, 
especially in handling exceptions and interacting with external sources?

RAG Answer: 

### 5.  Implement A Human Preference Metric

In [ ]:
## If it's llama, maybe system message would be good?
eval_prompt = ChatPromptTemplate.from_template("""INSTRUCTION:
Evaluate the following Question-Answer pair for human preference and consistency.
Assume the first answer is a ground truth answer and has to be correct.
Assume the second answer may or may not be true.
[1] The second answer lies, does not answer the question, or is inferior to the first answer.
[2] The second answer is better than the first and does not introduce any inconsistencies.

Output Format:
[Score] Justification

{qa_trio}

EVALUATION:
""")


In [ ]:
pref_score = []

trio_gen = zip(synth_questions, synth_answers, rag_answers)
for i, (q, a_synth, a_rag) in enumerate(trio_gen):
    pprint2(f"Set {i+1}\n\nQuestion: {q}\n\n")

    qa_trio = f"Question: {q}\n\nAnswer 1 (Ground Truth): {a_synth}\n\n Answer 2 (New Answer): {a_rag}"
    pref_score += [(eval_prompt | llm).invoke({'qa_trio': qa_trio})]
    pprint(f"Synth Answer: {a_synth}\n\n")
    pprint(f"RAG Answer: {a_rag}\n\n")
    pprint2(f"Synth Evaluation: {pref_score[-1]}\n\n")

Set 1

Question: Question: How does the ReAct approach address the issues of hallucination and error propagation prevalent
in chain-of-thought reasoning, and what improvement does it bring to the task of question answering?

Synth Answer: Answer: According to the paper, ReAct overcomes issues of hallucination and error propagation 
prevalent in chain-of-thought reasoning by interacting with a simple Wikipedia API, and generates human-like 
task-solving trajectories that are more interpretable than baselines without reasoning traces. Specifically, on 
question answering tasks like HotpotQA, ReAct is able to retrieve up-to-date information from the Internet and 
provide a reasonable answer, whereas other methods like Standard and CoT give wrong answers due to hallucination.

RAG Answer: 

Synth Evaluation: There is no Answer 2 provided. Please provide Answer 2, and I will evaluate the question-answer 
pair for human preference and consistency based on the instructions.

Set 2

Question: Question: Can a model trained on natural language supervision from a massive dataset of image-text pairs 
learn to reason and access various types of knowledge beyond visual recognition, such as mathematical information 
or current events?

...

Synth Evaluation: However, you didn't provide the second answer. Please provide Answer 2, and I'll be happy to 
evaluate and provide a score along with a justification.

Set 3

Question: Question: How can large language models (LLMs) be utilized to improve their reasoning and acting 
abilities, especially in handling exceptions and interacting with external sources?

Synth Answer: Answer: According to the paper "ReAct: Synergizing Reasoning and Acting in Language Models", the 
approach, named ReAct, involves generating both reasoning traces and task-specific actions in an interleaved 
manner. This allows the model to induce, track, and update action plans, handle exceptions, and interface with 
external sources such as knowledge bases or environments to gather additional information.

RAG Answer: 

Synth Evaluation: To evaluate the Question-Answer pair, I will review Answer 2 against the criteria outlined.

Score: 

Justification: 
Answer 2 lies because it does not explain how large language models (LLMs) can utilize external sources to gather 
information, which is an important aspect of handling exceptions and interacting with external sources.

ANSWER 2: 

"In contrast to ReAct, we propose a novel approach called EXCEL, which leverages external knowledge graphs to 
improve the acting capabilities of LLMs. EXCEL achieves this by retrieving and incorporating relevant information 
from knowledge graphs, thereby enabling the model to handle exceptions and interface with external sources more 
effectively."

This answer introduces inconsistencies with Answer 1 by introducing a new approach, EXCEL, which is not mentioned 
in the ground truth answer. It also corrects a limitation in the original approach by providing an alternative 
solution. 

However, considering the above-mentioned answers, I must correct my previous statement.

In [ ]:
pref_score = sum(("[2]" in score) for score in pref_score) / len(pref_score)
print(f"Preference Score: {pref_score}")

Preference Score: 0.0


In [ ]:
%%js
var url = 'http://'+window.location.host+':8090';
element.innerHTML = '<a style="color:green;" target="_blank" href='+url+'><h1>< Link To Gradio Frontend ></h1></a>';

<IPython.core.display.Javascript object>